In [ ]:
# Import bibliotek
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
from time import sleep

In [ ]:
# funkcja wykorzystana w kodach:

def zmiana_na_float (cecha):
    """ 
    Funkcja do zmiany danych poszczegolnych cech na float z odcięciem jednostki.
      
    Parametry:
    ----------
    cecha (str): Liczba podana jako ciąg znaków 
  
    Wynik:
    ------
    float: Ciąg znaków zamieniony na float.
    
    Przykład:
    ---------
    zmiana_na_float('50,2 m2')->50.2
      
    """
    
    return float(cecha.rsplit(' ',1)[0].replace(' ','').replace(',','.'))     #rsplit-rozdzielam po spacji, ale tylko jednej spacji licząc od końca. Jeśli w stringu będzie kilka spacji, to podzieli tylko po tej ostatniej   

## 1. Kod do pobrania szczegółów ogłoszeń (widocznych z poziomu strony głównej)

- najpierw nałożyłam filtr na Otodom, jakie nieruchomosci mnie interesują ( np. wg.lokalizacji, rodzaju nieruchomości - w tym wypadku mieszkania, powierzchni itd)
- po nałożeniu filtrów, Otodom zwrócił mi kilka stron z ogłoszeniami odpowiadającymi moim kryteriom
- w tym kodzie zaczytam jedynie dane z pierwszej strony zwróconej przez Otodom. Są to dane widoczne z poziomu tej strony ( bez zagłębiania się w szczegóły, które można zobaczyć tylko po wejściu w konkretne oferty)
- w dalszych kodach pobiorę dane z kolejnych stron zwróconych przez Otodom, jak również szczegółowe dane widoczne tylko po wejściu w konkretną ofertę 


In [ ]:
# pobieram pierwszą stronę z ofertami ( najpierw nałożyłam filtr na otodom i to jest link do odfiltrowanej strony)
adres_strony=input('Podaj adres strony www: ')
zawartosc = requests.get(adres_strony)

In [ ]:
# Tworzę obiekt BS, w którym są zawarte dane z wybranej strony www
soup = BeautifulSoup(zawartosc.content, 'html.parser')

In [ ]:
f = csv.writer(open('oferty_mieszkania.csv', 'w'))

In [ ]:
wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty wyszukane na otodom
wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert


for item in wszystkie_oferty_items: # dla każdej cechy, którą chcę wydobyć, szukam jej klasy na stronie Otodom
    naglowek=item.find(class_='text-nowrap hidden-xs').get_text()
    liczba_pokoi=item.find(class_="offer-item-rooms hidden-xs").get_text()
    powierzchnia=item.find(class_="hidden-xs offer-item-area").get_text()
    cena=item.find(class_="offer-item-price").get_text()
    cena_zlm2=item.find(class_="hidden-xs offer-item-price-per-m").get_text()
    strony_www=item.find_all('a')
    for item in strony_www:
        adres_www=item.get('href') ## pobieram link do każdej oferty
        
    # poniższe cechy zamieniam na float wg.wczesniej napisanej funkcji
    liczba_pokoi=zmiana_na_float(liczba_pokoi)
    powierzchnia=zmiana_na_float(powierzchnia)
    cena=zmiana_na_float(cena.strip())
    cena_zlm2 =zmiana_na_float(cena_zlm2)

    print('tytuł: ', naglowek)
    print('liczba_pokoi: ', liczba_pokoi)
    print('powierzchnia (m2): ', powierzchnia)
    print('cena (zł): ', cena)
    print('cena (zl_m2): ', cena_zlm2)
    print(adres_www, '\n')
    
    f.writerow([naglowek, liczba_pokoi,powierzchnia, cena, cena_zlm2, adres_www ]) ## zapisanie do csv

In [ ]:
## zapisanie do CSV
f.writerow(['tytuł', 'liczba_pokoi', 'powierzchnia','cena', 'cena_zlm2', 'adres_www' ])

## 2. Kod do pobrania szczegółów zawartych wewnątrz konkretnych ogłoszeń

- najpierw nałożyłam filtr na Otodom, jakie nieruchomosci mnie interesują ( np. wg.lokalizacji, powierzchni, rodzaju nieruchomości - w tym wypadku mieszkania, itd)
- po nałożeniu filtrów, Otodom zwrócił mi kilka stron z ogłoszeniami odpowiadającymi moim kryteriom
- w tym kodzie zaczytam dane z pierwszej strony zwróconej przez Otodom, przy czym są to dane szczegółowe widoczne po otwarciu konkretnych ofert
- po pobraniu danych, zapisuję je do pliku w formacie CSV
- w dalszych kodach pobiorę dane z kolejnych stron zwróconych przez Otodom

In [ ]:
# pobieram pierwszą stronę z ofertami ( najpierw nałożyłam filtr na otodom i to jest link do odfiltrowanej strona)
adres_strony=input('Podaj adres strony www: ')
zawartosc = requests.get(adres_strony)

In [ ]:
zawartosc.status_code

In [ ]:
# Tworzę obiekt BS, w którym są zawarte dane z wybranej strony www
soup = BeautifulSoup(zawartosc.content, 'html.parser')

In [ ]:
# kod do zaczytania szczegółów zawartych w ofertach (przy przeglądaniu strony www widoczne są tylko po wejściu w daną ofertę)
wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty wyszukane na otodom
wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert

for item in wszystkie_oferty_items:
    strony_www=item.find_all('a')
    for item in strony_www:
        adres_www=item.get('href') ## pobieram link do każdej oferty
        zawartosc2=requests.get(adres_www)
        soup2=BeautifulSoup(zawartosc2.content, 'html.parser')
        szczegoly=soup2.find_all(class_='article-offer')
        for item in szczegoly:       
            adres=item.find(class_='address-lane').get_text()
            powierzchnia=item.find(class_='big').get_text()
            cena=item.find(class_="box-price-value").get_text()
            try:# piętro nie zawsze jest podane w ofercie i wówczas zwraca błąd. Aby uniknąć zatrzymania kodu, wprowadziłam try 
                pietro=item.find(class_='param_floor_no').get_text()
            except AttributeError:
                pietro='brak informacji'
            liczba_pokoi=item.find(class_='room-lane').get_text()
            opis=item.find(itemprop='description').get_text()


        powierzchnia=zmiana_na_float(powierzchnia)   
        cena=zmiana_na_float(cena.strip())
        cena_zl_m2=round(cena/powierzchnia,2)
        liczba_pokoi=[int(s) for s in liczba_pokoi.split() if s.isdigit()]



    print('adres: ', adres.strip())
    print('powierzchnia (m2): ', powierzchnia)
    print('cena (zł): ', cena)
    print('cena (zł/m2): ', cena_zl_m2)
    print('pietro: ', pietro)
    print('liczba_pokoi: ', liczba_pokoi[0])
    print('opis: ', opis)
    print(adres_www, '\n')

    f.writerow([adres, powierzchnia,cena, cena_zl_m2, pietro, liczba_pokoi[0], opis ]) ## zapisanie do csv
        

##  3. Kod do pobrania ofert z kilku stron 

- najpierw nałożyłam filtr na Otodom, jakie nieruchomosci mnie interesują ( np. wg.lokalizacji, rodzaju nieruchomości - w tym wypadku mieszkania, powierzchni itd)
- po nałożeniu filtrów, Otodom zwrócił mi kilka stron z ogłoszeniami odpowiadającymi moim kryteriom
- w poprzednich kodach czytałam jedynie dane widoczne na pierwszej stronie zwrócoej przez Otodom, teraz zaczytam dane z wielu stron. Zrobię to na 2 sposoby:
    - w pierwszym kodzie użytkownik sam będzie podawał, z ilu stron chce uzyskać dane ( np. Otodom zwróci 10 stron, a użytkownik uzna, że chce dane tylko z 2 stron). Poza tym użytkownik będzie mógł wybrać, z których stron chce otrzymać dane, wpisując linki do tych stron
    - w drugim kodzie automatycznie zaczytam dane z wszystkich stron zwróconych przez Otodom
- dodatkowo uporządkuję pobrane dane wyorzystując bibliotekę Pandas (do csv będą eksportowane uporządkowane dane)


### 3a. Użytkownik podaje, z których stron chce pobrać dane

In [19]:
strony=[]
for strona in range(0,int(input("Podaj, ile stron chcesz przejrzeć: "))):
    strona=input(str("Wpisz pełen adres strony- cały link: "))
    strony.append(strona)
    print('strony: ', strony)

Podaj, ile stron chcesz przejrzeć: 1
Wpisz pełen adres strony- cały link: https://www.otodom.pl/sprzedaz/nowe-mieszkanie/wieliczka/?search%5Bfilter_float_m%3Ato%5D=39&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bsubregion_id%5D=163&search%5Bcity_id%5D=569
strony:  ['https://www.otodom.pl/sprzedaz/nowe-mieszkanie/wieliczka/?search%5Bfilter_float_m%3Ato%5D=39&search%5Bdescription%5D=1&search%5Bdist%5D=0&search%5Bsubregion_id%5D=163&search%5Bcity_id%5D=569']


In [ ]:
adresy=[]
powierzchnie=[]
ceny=[]
ceny_zlm2=[]
pietra=[]
pokoje=[]
dodatkowe=[]
opisy=[]

    
for item in strony:
    zawartosc = requests.get(item)
    soup=BeautifulSoup(zawartosc.content, 'html.parser')
    
    wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty
    wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert

    
    for item in wszystkie_oferty_items:
        strony_www=item.find_all('a')
        for item in strony_www:
            adres_www=item.get('href') ## pobieram link do każdej oferty
            zawartosc2=requests.get(adres_www)
            soup2=BeautifulSoup(zawartosc2.content, 'html.parser')
            szczegoly=soup2.find_all(class_='article-offer')
            for item in szczegoly:       
                adres=item.find(class_='address-lane').get_text()
                powierzchnia=item.find(class_='big').get_text()
                cena=item.find(class_="box-price-value").get_text()
                try:# piętro nie zawsze jest podane w ofercie i wówczas zwraca błąd. Aby uniknąć zatrzymania kodu, wprowadziłam try 
                    pietro=item.find(class_='param_floor_no').get_text()
                except AttributeError:
                    pietro='brak informacji'
                liczba_pokoi=item.find(class_='room-lane').get_text()
                opis=item.find(itemprop='description').get_text()
                dodatkowe_parametry=soup2.find_all(class_='sub-list')
                slownik={}
                for item in dodatkowe_parametry: 
                    wszystko=item.find_all('li')
                    for e in wszystko:
                        klucz,wartosc=e.text.split(':')
                        slownik[klucz]=wartosc
                      
                

                    powierzchnia=zmiana_na_float(powierzchnia)   
                    cena=zmiana_na_float(cena.strip())
                    cena_zl_m2=round(cena/powierzchnia,2)
                    liczba_pokoi=[int(s) for s in liczba_pokoi.split() if s.isdigit()]
                    

                    print('adres: ', adres.strip())# strip rozdziela ciąg znaków i zwraca listę 
                    print('powierzchnia (m2): ', powierzchnia)
                    print('cena (zł): ', cena)
                    print('cena (zł/m2): ', cena_zl_m2)
                    print('pietro: ', pietro)
                    print('liczba_pokoi: ', liczba_pokoi[0])
                    for klucz,wartosc in slownik.items():
                        print(klucz,': ',wartosc)
                    print('opis: ', opis)
                    print(adres_www, '\n')

                    # zapisuję wyciągnięte dane do takiej postaci, abym mogła je dalej zapisać w Pandasie
                    adresy.append(adres.strip())
                    powierzchnie.append(powierzchnia)
                    ceny.append(cena)
                    ceny_zlm2.append(cena_zl_m2)
                    pietra.append(pietro)
                    pokoje.append(liczba_pokoi[0])
                    opisy.append(opis)
                    dodatkowe.append(slownik)


### Obróbka  danych - Pandas

In [70]:
oferty=pd.DataFrame()
for x in range(0,len(adresy)):
    sr=pd.Series([adresy[x],powierzchnie[x],ceny[x],ceny_zlm2[x], pietra[x],pokoje[x], opisy[x] ],\
                 index=['adres','powierzchnia (m2)','cena (zł)','Cena (zł/m2)', 'pietro', 'liczba_pokoi', 'opis'])
    sr_dodatkowe=pd.Series(dodatkowe[x])
    sr=pd.concat([sr,sr_dodatkowe])
    oferty=pd.concat([oferty,sr],axis=1,ignore_index=True)

/home/kinga/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


In [71]:
oferty.index

Index(['Cena (zł/m2)', 'Forma własności', 'Okna', 'Rodzaj zabudowy',
       'Rok budowy', 'Rynek', 'adres', 'cena (zł)', 'liczba_pokoi', 'opis',
       'pietro', 'powierzchnia (m2)'],
      dtype='object')

In [72]:
oferty=oferty.T

In [73]:
oferty

,Cena (zł/m2),Forma własności,Okna,Rodzaj zabudowy,Rok budowy,Rynek,adres,cena (zł),liczba_pokoi,opis,pietro,powierzchnia (m2)
0,5749.99,pełna własność,plastikowe,apartamentowiec,2019,pierwotny,"Mieszkanie na sprzedaż, małopolskie, wielicki,...",201422,2,\nKAMERALNE OSIEDLE PRZY PARKU Z DOSKONAŁYM DO...,Piętro 3 (z 4),35.03
1,5549.99,pełna własność,plastikowe,apartamentowiec,2020,pierwotny,"Mieszkanie na sprzedaż, małopolskie, wielicki,...",212731,2,\nKAMERALNE OSIEDLE PRZY PARKU Z DOSKONAŁYM DO...,Piętro parter (z 4),38.33
2,5300,pełna własność,NaN,NaN,2018,pierwotny,"Mieszkanie na sprzedaż, małopolskie, wielicki,...",178504,2,\n0% PROWIZJI !!!OPIS MIESZKANIA:- Salon z ane...,Piętro 3 (z 5),33.68
3,5300,pełna własność,NaN,NaN,2018,pierwotny,"Mieszkanie na sprzedaż, małopolskie, wielicki,...",178504,2,\n0% PROWIZJI !!!!!!!!!!!!!!!!OPIS MIESZKANIA:...,Piętro 3 (z 5),33.68


In [74]:
kolejnosc_kolumn=['adres','Rynek','powierzchnia (m2)','cena (zł)','Cena (zł/m2)','Forma własności',\
'liczba_pokoi', 'pietro', 'Rodzaj zabudowy', 'Stan wykończenia', 'Czynsz', 'opis']

In [75]:
# ustawiam kolumny w wybranej kolejności. 
#Jeśli nieruchomość nie posiada danych w zakresie określonej cechy, w odpowiedniej komórce pojawi się 'Nan'
mieszkania_oferty=oferty.reindex(columns=kolejnosc_kolumn)

In [76]:
mieszkania_oferty

,adres,Rynek,powierzchnia (m2),cena (zł),Cena (zł/m2),Forma własności,liczba_pokoi,pietro,Rodzaj zabudowy,Stan wykończenia,Czynsz,opis
0,"Mieszkanie na sprzedaż, małopolskie, wielicki,...",pierwotny,35.03,201422,5749.99,pełna własność,2,Piętro 3 (z 4),apartamentowiec,NaN,NaN,\nKAMERALNE OSIEDLE PRZY PARKU Z DOSKONAŁYM DO...
1,"Mieszkanie na sprzedaż, małopolskie, wielicki,...",pierwotny,38.33,212731,5549.99,pełna własność,2,Piętro parter (z 4),apartamentowiec,NaN,NaN,\nKAMERALNE OSIEDLE PRZY PARKU Z DOSKONAŁYM DO...
2,"Mieszkanie na sprzedaż, małopolskie, wielicki,...",pierwotny,33.68,178504,5300,pełna własność,2,Piętro 3 (z 5),NaN,NaN,NaN,\n0% PROWIZJI !!!OPIS MIESZKANIA:- Salon z ane...
3,"Mieszkanie na sprzedaż, małopolskie, wielicki,...",pierwotny,33.68,178504,5300,pełna własność,2,Piętro 3 (z 5),NaN,NaN,NaN,\n0% PROWIZJI !!!!!!!!!!!!!!!!OPIS MIESZKANIA:...


In [79]:
# usuwam duplikaty
mieszkania_oferty.drop_duplicates(subset=['powierzchnia (m2)', 'cena (zł)','pietro','liczba_pokoi' ])

,adres,Rynek,powierzchnia (m2),cena (zł),Cena (zł/m2),Forma własności,liczba_pokoi,pietro,Rodzaj zabudowy,Stan wykończenia,Czynsz,opis
0,"Mieszkanie na sprzedaż, małopolskie, wielicki,...",pierwotny,35.03,201422,5749.99,pełna własność,2,3 (z 4),apartamentowiec,NaN,NaN,\nKAMERALNE OSIEDLE PRZY PARKU Z DOSKONAŁYM DO...
1,"Mieszkanie na sprzedaż, małopolskie, wielicki,...",pierwotny,38.33,212731,5549.99,pełna własność,2,parter (z 4),apartamentowiec,NaN,NaN,\nKAMERALNE OSIEDLE PRZY PARKU Z DOSKONAŁYM DO...
2,"Mieszkanie na sprzedaż, małopolskie, wielicki,...",pierwotny,33.68,178504,5300,pełna własność,2,3 (z 5),NaN,NaN,NaN,\n0% PROWIZJI !!!OPIS MIESZKANIA:- Salon z ane...


In [80]:
#usuwam wyraz'Piętro' z kolumny 'pietro'
mieszkania_oferty['pietro']=mieszkania_oferty['pietro'].apply(lambda x: x.replace('Piętro',''))

In [82]:
# usuwam 'escape characters' z kolumny 'opis'
mieszkania_oferty.loc[:,'opis'] = mieszkania_oferty.opis.str.replace('\n','',regex=True)

In [ ]:
mieszkania.to_csv(r'/home/kinga/python/Web_scrapping/mieszkania_oferty.csv')

### 3b. Kod, który automatycznie  pobiera dane ze wszystkich stron zwróconych przez Otodom

In [ ]:

strona=input('Wpisz adres strony internetowej: ')
strony=[strona]
page=requests.get(strona)
soup=BeautifulSoup(page.content, 'html.parser')

cala_strona=soup.find(class_='col-md-content section-listing__row-content') # dane całej strony
wiersz_dot_stron=cala_strona.find_all(class_='pager') # w tym jest klasa 'page-counter' i 'page_next'

przycisk_przewijania_stron=cala_strona.find_all(class_='pager-next')


for item in wiersz_dot_stron: # sprawdzam, ile stron z ogłoszeniami zostało znalezionych wg.wybranych kryteriów
    liczba_str=item.find(class_='current').get_text()
    print ('liczba stron: ', liczba_str)




In [ ]:
#i=1
for i in range(1,int(liczba_str)+1):
         
        strony.append(strona +'&page=' + str(i))  # każda kolejna strona ma końcówkę '&page=(nr_str)' w stosunku do strony pierwszej, poczynając od '&page=2' 

In [ ]:
print ('strony: ', strony)

In [ ]:
adresy=[]
powierzchnie=[]
ceny=[]
ceny_zlm2=[]
pietra=[]
pokoje=[]
dodatkowe=[]
opisy=[]

    
for item in strony:
    zawartosc = requests.get(item)
    soup=BeautifulSoup(zawartosc.content, 'html.parser')
    
    wszystkie_oferty = soup.find(class_='col-md-content section-listing__row-content') ### klasa, w ktorej są wszystskie oferty
    wszystkie_oferty_items = wszystkie_oferty.find_all(class_="offer-item-details")## klasa, w której są szczegóły poszczególnych ofert

    
    for item in wszystkie_oferty_items:
        strony_www=item.find_all('a')
        for item in strony_www:
            adres_www=item.get('href') ## pobieram link do każdej oferty
            zawartosc2=requests.get(adres_www)
            soup2=BeautifulSoup(zawartosc2.content, 'html.parser')
            szczegoly=soup2.find_all(class_='article-offer')
            for item in szczegoly:       
                adres=item.find(class_='address-lane').get_text()
                powierzchnia=item.find(class_='big').get_text()
                cena=item.find(class_="box-price-value").get_text()
                try:# piętro nie zawsze jest podane w ofercie i wówczas zwraca błąd. Aby uniknąć zatrzymania kodu, wprowadziłam try 
                    pietro=item.find(class_='param_floor_no').get_text()
                except AttributeError:
                    pietro='brak informacji'
                liczba_pokoi=item.find(class_='room-lane').get_text()
                opis=item.find(itemprop='description').get_text()
                dodatkowe_parametry=soup2.find_all(class_='sub-list')
                slownik={}
                for item in dodatkowe_parametry: 
                    wszystko=item.find_all('li')
                    for e in wszystko:
                        klucz,wartosc=e.text.split(':')
                        slownik[klucz]=wartosc
                      
                

                    powierzchnia=zmiana_na_float(powierzchnia)   
                    cena=zmiana_na_float(cena.strip())
                    cena_zl_m2=round(cena/powierzchnia,2)
                    liczba_pokoi=[int(s) for s in liczba_pokoi.split() if s.isdigit()]
                    

                    print('adres: ', adres.strip())# strip rozdziela ciąg znaków i zwraca listę 
                    print('powierzchnia (m2): ', powierzchnia)
                    print('cena (zł): ', cena)
                    print('cena (zł/m2): ', cena_zl_m2)
                    print('pietro: ', pietro)
                    print('liczba_pokoi: ', liczba_pokoi[0])
                    for klucz,wartosc in slownik.items():
                        print(klucz,': ',wartosc)
                    print('opis: ', opis)
                    print(adres_www, '\n')

                    # zapisuję wyciągnięte dane do takiej postaci, abym mogła je dalej zapisać w Pandasie
                    adresy.append(adres.strip())
                    powierzchnie.append(powierzchnia)
                    ceny.append(cena)
                    ceny_zlm2.append(cena_zl_m2)
                    pietra.append(pietro)
                    pokoje.append(liczba_pokoi[0])
                    opisy.append(opis)
                    dodatkowe.append(slownik)
